# Feature Engineering
- run once to generate dataset
- no comments and experiments, just debugging

In [1]:
# Modules import
from importlib import reload
import time
import pandas as pd
import numpy as np

# All special functions are written in support module
import support

In [2]:
reload(support)
print('Support Reloaded')

Support Reloaded


In [3]:
# Import data (we will immideatly merge some informaton together)
items              = pd.read_csv('./readonly/items.csv').set_index('item_id')
dataset            = pd.read_csv('./readonly/sales_train.csv')
test_dataset       = pd.read_csv('./readonly/test.csv').drop('ID', axis=1)
item_categories    = pd.read_csv('./readonly/item_categories.csv').set_index('item_category_id')
shops              = pd.read_csv('./readonly/shops.csv').set_index('shop_id')

In [4]:
dataset['item_price'] = dataset['item_price'].astype('float32')

for i in ['shop_id', 'item_id', 'date_block_num', 'item_cnt_day']:
    dataset[i] = dataset[i].astype('int32')

In [5]:
# Clear data
support.clear_records(dataset, test_dataset, shops, inplace=True)

# Group date num blocks
data_grouped = support.group_records(dataset)

# Choose validation block
validation_answers = data_grouped[data_grouped['date_block_num'] == 33]['target']
validation_answers.to_csv('./results/validation_answers.csv', index_label='index')

validation_test = data_grouped[data_grouped['date_block_num'] == 33].drop(['revenue', 'date_block_num'], axis=1).copy()

validation_dataset = dataset[dataset['date_block_num'] < 33].copy()

# Part for test
dataset = dataset.join(items[['item_category_id']], on='item_id')

item_mean_prices, shop_mean_prices, cat_mean_prices = support.get_prices_means(dataset)

full, train_mask = support.union(data_grouped, test_dataset)

full = full.join(items[['item_category_id']], on='item_id')
full = support.validation_preparation(full, train_mask, 10, 
                    [item_mean_prices, shop_mean_prices, cat_mean_prices])

full.to_csv('./results/dataset_for_test.csv', index=False)


# Part for validation
data_grouped = support.group_records(validation_dataset)

validation_dataset = validation_dataset.join(items[['item_category_id']], on='item_id')

item_mean_prices, shop_mean_prices, cat_mean_prices = support.get_prices_means(validation_dataset)

full, train_mask = support.union(data_grouped, validation_test)

full = full.join(items[['item_category_id']], on='item_id')
full = support.validation_preparation(full, train_mask, 10, 
                    [item_mean_prices, shop_mean_prices, cat_mean_prices])

full.to_csv('./results/dataset_for_validation.csv', index_label='index')

c:\users\shepe\python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Records are clear...
Zero sales generated...
Records collected by month...


c:\users\shepe\python\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  if __name__ == '__main__':


Mean prices are found...


100%|██████████████████████████████████████████| 10/10 [00:33<00:00,  3.36s/it]


Time-based features added...
Categories' features added...
Number of sales added...
Numerical features added...
Zero sales generated...
Records collected by month...
Mean prices are found...


100%|██████████████████████████████████████████| 10/10 [00:31<00:00,  3.14s/it]


Time-based features added...
Categories' features added...
Number of sales added...
Numerical features added...
